## Here, we are training a StyleGAN2 model from scratch to compare to StyleGAN2-ADA on the same dataset

In [19]:
from glob import glob
from tqdm import tqdm
from pathlib import Path
import zipfile
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/StyleGAN2-ADA')

import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

In [20]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR_, TRAIN_RUNS_DIR_ = DeepFAMS.utils.return_dirs(
    PROJ_DIR, 'FFHQ')

In [31]:
subset = ''

In [32]:
if subset != '':
    subset = '_' + subset.upper()

DATA_CUSTOM_DIR = f'{Path(DATA_CUSTOM_DIR_)}' + subset
TRAIN_RUNS_DIR = f'{Path(TRAIN_RUNS_DIR_).parent}/StyleGAN2_{Path(TRAIN_RUNS_DIR_).name}' + subset

In [ ]:
# ! $WORK/.conda/envs/ada-env/bin/kaggle datasets download -d \
#     arnaud58/flickrfaceshq-dataset-ffhq -p $WORK/ADA_Project/datasets

In [ ]:
# downloaded_file = f'{Path(RAW_IMGS_DIR).parent}/flickrfaceshq-dataset-ffhq.zip'

# with zipfile.ZipFile(downloaded_file) as zf:    
#     for member in tqdm(zf.infolist(), desc='Extracting'):
#         try:
#             zf.extract(member, RAW_IMGS_DIR)
#         except zipfile.error as e:
#             pass

In [33]:
raw_imgs = glob(f'{RAW_IMGS_DIR}/*')
print(len(raw_imgs))

52001


In [34]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

In [35]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 52001, Resized: 52001


In [ ]:
# DeepFAMS.preprocessing.tf_record_exporter(
#     tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

In [36]:
! nvidia-smi

Fri Apr 23 14:39:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   27C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [37]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/6.1
module load cuda/10.0
$WORK/.conda/envs/stylegan2/bin/python3 $WORK/ADA_Project/stylegan2/run_training.py \
    --num-gpus=2 \
    --data-dir=$WORK/ADA_Project/datasets \
    --result-dir={TRAIN_RUNS_DIR} \
    --config=config-f \
    --mirror-augment=true \
    --total-kimg=1 \
    --dataset={Path(DATA_CUSTOM_DIR).name}''', PROJ_DIR)

Local submit - run_dir: /work/chaselab/malyetama/ADA_Project/training_runs/StyleGAN2_FFHQ_training-runs/00000-stylegan2-FFHQ_custom-2gpu-config-f

dnnlib: Running training.training_loop.training_loop() on localhost...

Streaming data using training.dataset.TFRecordDataset...

Dataset shape = [3, 256, 256]

Dynamic range = [0, 255]

Label size    = 0

Constructing networks...

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.

Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.



G                             Params    OutputShape         WeightShape     

---                           ---       ---                 ---             

latents_in                    -         (?, 512)            -               

labels_in                     -         (?, 0)              -               

lod                           -         ()                  -               

dlatent_avg                   -      

KeyboardInterrupt: 

In [ ]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

In [ ]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/6.1
module load cuda/10.0
$WORK/.conda/envs/stylegan2/bin/python3 $WORK/ADA_Project/stylegan2/run_training.py \
    --num-gpus=2 \
    --data-dir=$WORK/ADA_Project/datasets \
    --result-dir={TRAIN_RUNS_DIR} \
    --config=config-f \
    --mirror-augment=true \
    --dataset={Path(DATA_CUSTOM_DIR).name}
    --resume-pkl={latest_snap}''', PROJ_DIR)